In [1]:
import os
import cv2 as cv
import numpy as np
from PIL import Image
import torch
from torch import nn
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [3]:
def load_images_from_folder(folder):
    images = []
    for file in os.listdir(folder):
        img = cv.imread(os.path.join(folder,file))
        if(img is not None):
            images.append(img)
    return images

In [4]:
def resize_and_flatten(img, target_size=(64,64)):
    resized_img = cv.resize(img, target_size)
    flattened_img = resized_img.reshape(target_size[0] * target_size[1] * 3)
    return flattened_img

In [5]:
cat_data = []
for img in load_images_from_folder("../data/cat"):
    cat_data.append(resize_and_flatten(img))
dog_data = []
for img in load_images_from_folder("../data/dog"):
    dog_data.append(resize_and_flatten(img))

In [6]:
print(str(len(cat_data)) + " cat images")
print(str(len(dog_data)) + " dog images")

80 cat images
80 dog images


In [7]:
X = []
y = []

for entity in cat_data:
    X.append(entity)
    y.append(1.0)

for entity in dog_data:
    X.append(entity)
    y.append(0.0)


In [8]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(64*64*3, 16*16),
            nn.ReLU(),
            nn.Linear(16*16, 16*16),
            nn.ReLU(),
            nn.Linear(16*16,1)
        )
    def forward(self, x, **kwargs):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
model = MyNN()

In [10]:
net = NeuralNetClassifier(
    module = model,
    max_epochs=10,
    lr=0.001,
    iterator_train__shuffle=True,
)

In [11]:
params = {    
    'lr': [0.001, 0.01, 0.0001],
    'max_epochs': [1,2,5, 10, 20, 25],
}

gs = GridSearchCV(net, params, cv=2)
gs.fit(torch.FloatTensor(X), torch.FloatTensor(y))
pd.DataFrame(gs.cv_results_)

/var/folders/x_/gy3_q9jx56qc789tf8h0bhv00000gn/T/ipykernel_1575/2296238450.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  gs.fit(torch.FloatTensor(X), torch.FloatTensor(y))


ValueError: 
All the 36 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
36 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/gbecker/git/bachelor-thesis/notebooks/myfirstNN/venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/gbecker/git/bachelor-thesis/notebooks/myfirstNN/venv/lib/python3.9/site-packages/skorch/classifier.py", line 165, in fit
    return super(NeuralNetClassifier, self).fit(X, y, **fit_params)
  File "/Users/gbecker/git/bachelor-thesis/notebooks/myfirstNN/venv/lib/python3.9/site-packages/skorch/net.py", line 1319, in fit
    self.partial_fit(X, y, **fit_params)
  File "/Users/gbecker/git/bachelor-thesis/notebooks/myfi

In [ ]:
test_imgs = load_images_from_folder("../notebooks/myfirstNN/data/cat")